<a href="https://colab.research.google.com/github/arcila-heisman/ml_introductory_labs/blob/master/Lab_4/Laboratorio_4_parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

Modificado por Heisman Arcila. (Versión original en: https://github.com/jdariasl/ML_IntroductoryCourse/tree/master/Labs)

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Danny Francisco Hernandez Godoy
#### CC: 1105784633
#### Segundo Integrante: Daniel Martinez Galeano
#### CC: 1036645337

In [0]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("ignore")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

In [0]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

## Ejercicio 1

Complete el script siguiente con el código necesario para cargar los datos en el archivo 'DB/data_UCI.csv', y luego usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

Para este ejercicio usaremos la base de datos sobre la estabilidad de una red eléctrica distribuída, en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls '/gdrive/My Drive/simulacion DMG/DB'

ls: cannot access '/gdrive/My Drive/simulacion DMG/DB': No such file or directory


In [0]:
#cargue la bd que está en un archivo .csv y ahora la podemos manejar de forma matricial
db = np.loadtxt('/gdrive/My Drive/2019-1/Simulación/Lab4/DB/data_UCI.csv',delimiter=',', skiprows=1)
#Esta es la base de datos Electrical Grid Stability Simulated Data Data Set del UCI Machine Learning Repository. 
#En la siguiente URL se encuentra toda la descripción de la base de datos y la contextualización del problema.
#En éste caso se ha mapeado a 0 la red inestable y a 1 la red estable en la columna stabf
#https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data

X = db[:,0:11]
Y = db[:,11:13]

In [0]:
print(np.unique(Y))

[-0.08075989 -0.07492328 -0.07052886 ...  0.9997388   0.99988163
  0.99993005]


In [0]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
import numpy.matlib as matlib

  
def invento(neuronas):
  #Validamos el modelo
  Folds = 4
  random.seed(19680801)
  ErrorY1 = np.zeros(Folds)
  ErrorY2 = np.zeros(Folds)
  ErrorT = np.zeros(Folds)
  ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
  j = 0
  for train, test in ss.split(X):
      Xtrain = X[train,:]
      Ytrain = Y[train,:]
      Xtest = X[test,:]
      Ytest = Y[test,:]

      #Normalizamos los datos
      media = np.mean(Xtrain,axis=0)
      desvia = np.std(Xtrain,axis=0)
      Xtrain = preprocessing.scale(Xtrain)
      Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
      mlp = MLPRegressor(hidden_layer_sizes=neuronas, activation='tanh', max_iter=500)
      mlp.fit(Xtrain, Ytrain)
      activacion=mlp.out_activation_

      #Use para el modelo para hacer predicciones sobre el conjunto Xtest
      Yest = mlp.predict(Xtest)

      #Mida el error MAPE para cada una de las dos salidas
      ErrorY1[j] = MAPE(Yest[:, 0], Ytest[:, 0])
      ErrorY2[j] = MAPE(Yest[:, 1], Ytest[:, 1])
      ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
      j += 1

  print('MAPE salida 1 = ' + str(np.mean(ErrorY1)) + '+-' + str(np.std(ErrorY1)))
  print('MAPE salida 2 = ' + str(np.mean(ErrorY2)) + '+-' + str(np.std(ErrorY2)))
  print('MAPE total = '    + str(np.mean(ErrorT)) + '+-' + str(np.std(ErrorT)))
  
  return np.mean(ErrorY1), np.std(ErrorY1), np.mean(ErrorY2), np.std(ErrorY2), activacion

## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [0]:
import pandas as pd
#import qgrid
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,(20,20),(24,24),(28,28),(32,32),(36,36)])})
    #https://stackoverflow.com/questions/35363530/python-scikit-learn-mlpclassifier-hidden-layer-sizes
df_types["MAPE salida 1"] = ""
df_types["IC MAPE salida 1"] = ""
df_types["MAPE salida 2"] = ""
df_types["IC MAPE salida 2"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

for i in df_types.index:
    print(i[0])
    ED=invento(i[1])
    df_types["MAPE salida 1"][i] = float(ED[0])
    df_types["IC MAPE salida 1"][i] = float(ED[1])
    df_types["MAPE salida 2"][i] = float(ED[2])
    df_types["IC MAPE salida 2"][i] = float(ED[3])
#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

1


NameError: ignored

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

#No Funciona QGRID

In [0]:
#qgrid_widget.get_changed_df()
df_types

MAPE salida 1  ... IC MAPE salida 2
N. de capas ocultas Neuronas por capa                ...                 
1                   20                     0.941818  ...           1.8121
                    24                     0.936142  ...          32.8599
                    28                     0.942167  ...          9.57071
                    32                     0.930984  ...         0.551738
                    36                     0.904492  ...          4.94593
2                   (20, 20)               0.929298  ...          5.58691
                    (24, 24)               0.943869  ...          4.72988
                    (28, 28)               0.918009  ...          3.81766
                    (32, 32)                0.91944  ...          4.49966
                    (36, 36)               0.953176  ...          1.69082

[10 rows x 4 columns]

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?:

In [0]:
activation=invento(20)
print('Funcion de activacion '+activation[4])

MAPE salida 1 = 0.9418175983663336+-0.006484875456104206
MAPE salida 2 = 4.309020830852207+-1.8120969483874103
MAPE total = 2.62541921460927+-0.9058130136425651
Funcion de activacion identity


## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación imagenes de prendas de vestir. 
Usaremos 10 clases disponibles. 
Los datos fueron preprocesados para cambiar su dimensionalidad a una adecuada para el problema.

https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles

In [0]:
from keras import datasets
(x, y), (_, _) = datasets.fashion_mnist.load_data()
X = x.reshape(x.shape[0],x.shape[1]*x.shape[2])[:2000,:]
Y = y[:2000]

Using TensorFlow backend.


4423680/4422102 [==============================] - 0s 0us/step


Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

def invento2(neuronas):
  Folds = 4
  random.seed(19680801)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  for train, test in skf.split(X, Y):
      Xtrain = X[train,:]
      Ytrain = Y[train]
      Xtest = X[test,:]
      Ytest = Y[test]

      #Normalizamos los datos
      media = np.mean(Xtrain)
      desvia = np.std(Xtrain)
      Xtrain = preprocessing.scale(Xtrain)
      Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
      mlp = MLPClassifier(hidden_layer_sizes=neuronas, activation='tanh', max_iter=500)
      mlp.fit(Xtrain, Ytrain)
      salida=mlp.n_outputs_
      activacion=mlp.out_activation_

      #Validación con las muestras de entrenamiento
      Ytrain_pred = mlp.predict(Xtrain)

      #Validación con las muestras de test
      Yest = mlp.predict(Xtest)

      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
      EficienciaVal[j] = np.mean(Yest == Ytest)
      j += 1

  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  return np.mean(EficienciaVal), np.std(EficienciaVal), str(salida), str(activacion)

## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [0]:
import pandas as pd
df_types2 = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,(20,20),(24,24),(28,28),(32,32),(36,36)])})
df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
"""
  df_types["Eficiencia en validacion"][2,24] = 0.702417	
  df_types["Intervalo de confianza"][2,24] = 0.027835
"""
#Ponga acá su código, use como valores de referencia los valores anteriores para comprobar su código
for i in df_types2.index:
    print(i[0])
    EX=invento2(i[1])
    df_types2["Eficiencia en validacion"][i] = float(EX[0])
    df_types2["Intervalo de confianza"][i] = float(EX[1])


#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

1
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7116046796369084+-0.044214734191810094
1
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.705531860978191+-0.015691850907400655
1
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7095712742588387+-0.022117406361321423
1
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7304475308403497+-0.052215833326891205
1
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7115364193579294+-0.026800239895224655
2
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.6885698859054508+-0.07434193243565398
2
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7024167623807613+-0.027835027061355537
2
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7111204670860702+-0.030788424518184598
2


Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.
#No Funciona QGRID

In [0]:
#qgrid_widget.get_changed_df()
df_types2

Eficiencia en validacion Intervalo de confianza
N. de capas ocultas Neuronas por capa                                                
1                   20                                0.711605              0.0442147
                    24                                0.705532              0.0156919
                    28                                0.709571              0.0221174
                    32                                0.730448              0.0522158
                    36                                0.711536              0.0268002
2                   (20, 20)                           0.68857              0.0743419
                    (24, 24)                          0.702417               0.027835
                    (28, 28)                           0.71112              0.0307884
                    (32, 32)                          0.716771              0.0499604
                    (36, 36)                          0.735465               0.040296

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

Función softmax ya que la salidas estan clasificadas con 0 y 1

4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?
Numero de neuronas en la capa de salida son 10, ya que estas 10 proporcionan todas las salidas de clasificacón.

In [0]:
respuesta=invento2(20)
print('Funcion de activación '+respuesta[3])
print('Neuronas en la capa de salida '+respuesta[2])

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.7116046796369084+-0.044214734191810094
Funcion de activación softmax
Neuronas en la capa de salida 10
